# Introduction to Feature View and Model Registry in Hopsworks
In this notebook we will join our previously created feature groups into a single feature view, create a train, test split dataset and train a simple model Gradient Boosting Classifier where we will want to predict the chances or of survival of passengers of the Titanic depending on their demographics and socio-economic status.

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
import hopsworks
from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import seaborn as sns
import os
import joblib

In [ ]:
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
titanic_passengers_fg = fs.get_feature_group(name="titanic_passengers",version=1)
titanic_tickets_fg = fs.get_feature_group(name="titanic_tickets",version=1)
titanic_survived_fg = fs.get_feature_group(name="titanic_survived",version=1)

In [ ]:
query = titanic_passengers_fg.select_except(["name"]).join(
    titanic_tickets_fg.select_all()).join(titanic_survived_fg.select_all())

In [ ]:
try:
    fv = fs.get_feature_view(name="titanic", version=1)
except:
    fv = fs.create_feature_view(name="titanic", 
                                version=1, 
                                query=query, 
                                labels=["survived"])

In [ ]:
X_train, X_test, y_train, y_test = fv.train_test_split(test_size=0.2)
X_train

In [ ]:
y_train

In [ ]:
# Train our model with the Scikit-learn binary classifier algorithm using our features (X_train) and labels (y_train)
model = GradientBoostingClassifier(n_estimators=1000, random_state=42)
model.fit(X_train, y_train.values.ravel())

In [ ]:
# Evaluate model performance using the features from the test set (X_test)
y_pred = model.predict(X_test)

# Compare predictions (y_pred) with the labels in the test set (y_test)
metrics = classification_report(y_test, y_pred, output_dict=True)
results = confusion_matrix(y_test, y_pred)
results

In [ ]:
# Create the confusion matrix as a figure, we will later store it as a PNG image file
df_cm = pd.DataFrame(results, ['True Dead', 'True Survived'], 
                                ['Predicted Dead', 'Predicted Survived'])

cm = sns.heatmap(df_cm, annot=True)
fig = cm.get_figure()
fig.show()

In [ ]:
# We will now upload our model to the Hopsworks Model Registry. First get an object for the model registry.
mr = project.get_model_registry()

# The contents of the 'titanic_model' directory will be saved to the model registry. Create the dir, first.
model_dir="titanic_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

# Save both our model and the confusion matrix to 'model_dir', whose contents will be uploaded to the model registry
joblib.dump(model, model_dir + "/titanic_model.pkl")
fig.savefig(model_dir + "/confusion_matrix.png")    


# Specify the schema of the model's input/output using the features (X_train) and labels (y_train)
input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema, output_schema)

# Create an entry in the model registry that includes the model's name, desc, metrics
titanic_model = mr.python.create_model(
    name="titanic_model", 
    metrics={"accuracy" : metrics['accuracy']},
    model_schema=model_schema,
    description="titanic Predictor"
)

# Upload the model to the model registry, including all files in 'model_dir'
titanic_model.save(model_dir)

In [ ]:
%%writefile predict_example.py

import joblib
import os

class Predict(object):
    
    def __init__(self):
        # NOTE: env var ARTIFACT_FILES_PATH has the local path to the model artifact files        
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/titanic_model.pkl")


    def predict(self, inputs):
        """ Serves a prediction request from a trained model"""
        return self.model.predict(inputs).tolist()